# Wildfire Forecasting: Quantile Regression on Fire Size

This notebook can be used for training a regression model for forecasting wildfire size in hectares. It is intended to be a step-by-step tutorial. We are using the 1 degree version of the [SeasFire Datacube](https://zenodo.org/record/7108392). You can also use the 0.25 degree version, but you might get out of memory errors. The datacube has to be downloaded first. It has many different variables that are all described on the zenodo page. For simplicity sake, we use only a subset. 

This tutorial is intended to be run on Google Colab with the data stored in Google Drive. You might need to adjust some paths etc. in order to run it locally.

🔥 
**Here you can define which variables from the datacube you want and which datacube you want, into how many quantiles you want to divide the fire sizes. You can also choose the target variable and if you want to train with unmasked or masked loss.**

In [ ]:
NO_QUANTILES = 25
DEG_DATACUBE = 1
LOSS = "masked"
DELTA_TIME = 2
VARIABLES_SELECTED = ['rel_hum', 'ndvi', 'pop_dens_LOG',
                      'tp_LOG', 'sst', 'gwis_ba', 'lst_day']
TARGET = 'gwis_ba'

## Imports

In [ ]:
# from utils.plot import visualize_batch_prediction
# from utils.dataloader import create_datasets_model
# from utils.general import seed_everything
# from utils.model import UNet

In [ ]:
train_on_gpu = torch.cuda.is_available()
print(f"GPU available? {train_on_gpu}")
SEED = 172
MODEL_NO = 0 # in K-fold
N_FOLDS = 10 # in K-fold
seed_everything(SEED)

In [ ]:
%%capture 
import os
import shutil
import gc
import cv2
import json 
import time
import tqdm
import random
import collections
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
from functools import partial
import matplotlib.pyplot as plt
from tqdm.auto import tqdm as tq
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from tqdm.notebook import tqdm as ntqdm

from typing import List
import scipy
from scipy.stats import boxcox

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau

import albumentations as albu

plt.style.use('bmh')

## Load data set

First, we need to load the dataset and perform some calculations for normalising the data. We also calculate the quantiles here.

🔥 **You might need to adjust some paths to reflect your file structure here!**

In [ ]:
%%capture
from google.colab import drive
drive.mount('/content/drive')
!pip install zarr
!pip install shapely cartopy --no-binary shapely --no-binary cartopy
!pip install --upgrade geopandas pyshp shapely


import geopandas as gpd
import xarray as xr
from pathlib import Path

if DEG_DATACUBE == 1:
  ds = xr.open_zarr('/content/drive/MyDrive/seasfire_1deg.zarr') #edit to your path
else:
  ds = xr.open_zarr('/content/drive/MyDrive/seasfire.zarr').     #edit to your path
mask = xr.where(ds.lsm>=0.9,1,0).astype(bool)#


# put the log for population_density and total_precipitation (there are too many 0 entries)
ds = ds.assign(pop_dens_LOG = lambda x: np.log(1+x['pop_dens']))
ds = ds.assign(tp_LOG = lambda x: np.log(1+x['tp']))

The data used has very different distributions (e.g. temperatures are given in Kelvin, but the Normalized Difference Vegetation Index is only defined in the range (-1, 1). In order to make it easier for the model to learn, we normalise all data to have a mean of 0, and a standard deviation of 1. The normalisation itself is done in the dataloader, but we calculate the means and standard deviations here.

🔥 **You might need to adjust some paths to reflect your file structure here!**

In [ ]:
file_mean_std = f'/content/drive/MyDrive/challenge - NOA - XAI for Wildfire Forecasting/Notebooks/develop_Johanna/table_mean_std_{DEG_DATACUBE}deg_{"_".join(VARIABLES_SELECTED)}.csv'

if not Path(file_mean_std).exists():
  ### SAVE IN A TABLE GLOBAL MEAN AND STD for EACH VARIABLE
  vmean = ds.mean()[VARIABLES_SELECTED].load().to_pandas()
  vmean.name = 'mean'
  vstd  = ds.std() [VARIABLES_SELECTED].load().to_pandas()
  vstd.name  = 'std'
  table_mean_std = pd.concat([vmean, vstd], axis=1)
  table_mean_std.to_csv(file_mean_std)
else:
  table_mean_std = pd.read_csv(file_mean_std, index_col = 0, header = 0)

table_mean_std

The raw values of the burned area in hectares have a quite large range from `0` to almost `1,000,000` hecatares in a `1 degree x 1 degree` pixel. We calculate the quantiles of fire sizes excluding 0 values and NaNs in the sea surface (otherwise, the data is too skewed).

🔥 **You might need to adjust some paths to reflect your file structure here!**
🔥

In [ ]:
file_fire_quantiles = f'/content/drive/MyDrive/challenge - NOA - XAI for Wildfire Forecasting/Notebooks/develop_Johanna/fire_quantiles_{DEG_DATACUBE}deg_{NO_QUANTILES}.csv'
quantiles = np.linspace(0, 1, num=NO_QUANTILES)
if not Path(file_fire_quantiles).exists():
  gwis_array = np.asarray(ds.gwis_ba)
  quantiles_gwis = np.nanquantile(gwis_array[gwis_array>0.0], quantiles)
  fcci_array = np.asarray(ds.fcci_ba)
  quantiles_fcci = np.nanquantile(fcci_array[fcci_array>0.0], quantiles)

  fire_quantiles = pd.DataFrame([list(quantiles_fcci), list(quantiles_gwis)], ["fcci_ba", "gwis_ba"], [quantiles])
  fire_quantiles.to_csv(file_fire_quantiles)

else:
  fire_quantiles = pd.read_csv(file_fire_quantiles, index_col = 0, header = 0)

fire_quantiles

Build a custom Dataset Class (following [PyTorch advices](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files))

🔥 **Here the quantiles are assigned to the target in __getitem__() (if self.fire_quantiles is not None)**

What I am trying to do is this, imagine if these are the burned area values that we have in the whole dataset:

1.   I start from the raw burned area values (without NaNs and 0s), e.g. **[1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 5, 6, 7, 8, 8, 9, 10]**
2.   I want e.g. 5 quantiles  so the **0, 0.25, 0.5, 0.75 and 1** quantiles!
3. I calculate the values for the quantiles: **1, 1.75, 3, 6.25, 10**

In the **itemgetter** function I then change the burned areas to the enumerated quantile they are in (NaNs are turned into -1 and 0 stay 0) So e.g.:

`[[0, 0, NaN], [1, 1, 2], [5, 6, 8]]` would be turned into:
`[[0, 0, -1], [0.25, 0.25, 0.5], [0.75, 1, 1]]`


In [ ]:
ds = ds.chunk(dict(time = len(ds.time),
                   latitude = 90, 
                   longitude = 90))

train_dataset, valid_dataset, test_dataset = create_datasets_model(ds[VARIABLES_SELECTED], 
                                                                   slice_train = slice('20030101', '20161231'),
                                                                   slice_valid = slice('20170101', '20181231'), 
                                                                   slice_test  = None,
                                                                   table_mean_std = table_mean_std,
                                                                   fire_quantiles = fire_quantiles,
                                                                   target='gwis_ba', delta_time = DELTA_TIME, inland_map = mask)

## Model Definition

U-Net was originally invented and first used for biomedical image segmentation (here the [original paper](https://arxiv.org/pdf/1505.04597.pdf)). Its architecture can be broadly thought of as an encoder network (contraction block), a bottleneck, followed by a decoder network (expansion section).

I you wish to deep dive into U-Net and Image Segmentation, here a [detailed blog post](https://www.jeremyjordan.me/semantic-segmentation/).

In [ ]:
model = UNet(n_channels=len(train_dataset.features), 
             n_classes=1,
             regression=True).float()
             
if train_on_gpu:
    model.cuda()

## Training Loop

In [ ]:
num_workers = 0
bs = 4

# DataLoader to batch the images
train_loader = DataLoader(
    train_dataset, batch_size=bs, shuffle=True, num_workers=num_workers)

valid_loader = DataLoader(
    valid_dataset, batch_size=bs, shuffle=False, num_workers=num_workers)

In [ ]:
from torchsummary import summary
from torch.nn import MSELoss
import warnings

#ignore some deprecation warnings
warnings.filterwarnings("ignore", category=UserWarning) 

model = UNet(n_channels=len(train_dataset.features), n_classes=1, regression=True).float()
if train_on_gpu:
    model.cuda()

In [ ]:
train_loss_list, valid_loss_list, dice_score_list, lr_rate_list, valid_loss_min = model.train_model(
                                                                                          train_loader = train_loader,
                                                                                          valid_loader = valid_loader,
                                                                                          n_epochs = 32,
                                                                                          t_mask = None,
                                                                                          criterion = MSELoss(), #Dice is used for segmentation
                                                                                          optimizer = optim.Adam(model.parameters(), lr = 0.005),
                                                                                          scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.2, patience=2, cooldown=2)
                                                                                      )

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(train_loss_list,  marker='o', label="Training Loss")
plt.plot(valid_loss_list,  marker='o', label="Validation Loss")
plt.ylabel('loss', fontsize=22)
plt.legend()
plt.show()